In [105]:
#Kuhn Poker is a 3 action, partial information game
#This program implements CFR for one player

In [106]:
PASS = 0
BET = 1
NUM_ACTIONS = 2

In [107]:
from random import seed
from random import random
from random import shuffle
seed(42)

In [108]:
from blist import sorteddict

In [109]:
#SortedDict is close to TreeMap class from Java
nodeMap = sorteddict()

#Node definitions
infoSet = ""
regretSum = [0.0] * NUM_ACTIONS
strategy  = [0.0] * NUM_ACTIONS
strategySum = [0.0] * NUM_ACTIONS
print(regretSum)

#This function returns the current strategy of a given trainer
def getStrategy(realizationWeight):
    normalizingSum = 0.0
    for a in range(NUM_ACTIONS):
        strategy[a] = regretSum[a] if (regretSum[a] > 0.0) else 0.0
        normalizingSum += strategy[a]
    for a in range(NUM_ACTIONS):
        if(normalizingSum > 0):
            strategy[a] /= normalizingSum
        else:
            strategy[a] = 1.0 / NUM_ACTIONS
        strategySum[a] += realizationWeight * strategy[a]
    return strategy

In [110]:
def getAverageStrategy():
    avgStrategy = [0.0] * NUM_ACTIONS
    normalizingSum = 0.0
    for a in range(NUM_ACTIONS):
        normalizingSum += strategySum[a]
    for a in range (NUM_ACTIONS):
        if(normalizingSum > 0):
            avgStrategy[a] = strategySum[a] / normalizingSum
        else:
            avgStrategy[a] = 1.0 / NUM_ACTIONS
    return avgStrategy

In [111]:
def toString():
    return '{:4}: {}'.format(infoSet, str(getAverageStrategy()))
toString()

'    : [0.5, 0.5]'

In [112]:
class Node:
    #Node definitions
    infoSet = ""
    regretSum = [0.0] * NUM_ACTIONS
    #strategy  = [0.0] * NUM_ACTIONS
    strategySum = [0.0] * NUM_ACTIONS
    nodeMap = sorteddict()
    #Get current information set mixed strategy through regret-matching
    def getStrategy(self,realizationWeight):
        normalizingSum = 0.0
        for a in range(NUM_ACTIONS):
            strategy[a] = regretSum[a] if (regretSum[a] > 0.0) else 0.0
            normalizingSum += strategy[a]
        for a in range(NUM_ACTIONS):
            if(normalizingSum > 0):
                strategy[a] /= normalizingSum
            else:
                strategy[a] = 1.0 / NUM_ACTIONS
            strategySum[a] += realizationWeight * strategy[a]
        return strategy
    #get average information set mixed strategy across all training iterations
    def getAverageStrategy():
        avgStrategy = [0.0] * NUM_ACTIONS
        normalizingSum = 0.0
        for a in range(NUM_ACTIONS):
            normalizingSum += strategySum[a]
        for a in range (NUM_ACTIONS):
            if(normalizingSum > 0):
                avgStrategy[a] = strategySum[a] / normalizingSum
            else:
                avgStrategy[a] = 1.0 / NUM_ACTIONS
        return avgStrategy
    #get information set string representation
    def toString(self):
        return '{:4}: {}'.format(infoSet, str(getAverageStrategy()))

In [113]:
def cfr(cards, history, p0,p1):
    plays = len(history)
    player = plays % 2
    opponent = 1 - player
    #Return payoff for terminal states
    if(plays > 1):
        terminalPass = history[plays - 1] == 'p'
        doubleBet = history[plays-2:plays] == 'bb'
        isPlayerCardHigher = cards[player] > cards[opponent]
        if terminalPass:
            if(history == 'pp'):
                return 1 if isPlayerCardHigher else -1
            else:
                return 1
        elif (doubleBet):
            return 2 if isPlayerCardHigher else -2
    infoSet = str(cards[player]) + history
    #get information set node or create if nonexistant
    node = nodeMap.get(infoSet)
    if(node == None):
       node = Node()
       node.infoSet = infoSet
       nodeMap[infoSet] = node
    #For each action, recursively call cfr with additional history and probability
    strategy = node.getStrategy(p0 if (player == 0 ) else p1)
    util = [0.0] * NUM_ACTIONS
    nodeUtil = 0
    for a in range(NUM_ACTIONS):
       nextHistory = history + ('p' if (a == 0) else 'b')
       util[a] = -cfr(cards,nextHistory,p0 * strategy[a],p1) if player == 0 else -cfr(cards,nextHistory,p0,p1 * strategy[a])
       nodeUtil += strategy[a] * util[a]
    #For each action, compute and accumulate counterfactual regret
    for a in range(NUM_ACTIONS):
       regret = util[a] - nodeUtil
       node.regretSum[a] += (p1 if (player == 0) else p0) * regret
    
   
    return nodeUtil

In [114]:
def train(iterations):
    cards = [1,2,3]
    util = 0.0
    for i in range(iterations):
        shuffle(cards)
        util += cfr(cards,"",1,1)
    print("Average game value: " + str(float(util / iterations)))
    for a in nodeMap.values():
        print(a.toString())

In [ ]:
train(1000000)